## Exploring Predictors
After identifying some of the predictors that we think might be useful in helping us classify our wallets, we want to explore which combination of these 5 predictors would be most useful.

We decided to run some loops, and compare the classification accuracies of the different combination of predictors, of each classification model.

In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

# Import essential models and functions from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

## Decision Tree Classifier

In [15]:
def classify(xdata, ydata): 

    # Split the dataset into Train and Test
    x_train, x_test, y_train, y_test = train_test_split(xdata, ydata, test_size = 0.2)

    dectree1 = DecisionTreeClassifier(max_depth = 5)
    dectree1.fit(x_train, y_train)

    y_test_pred1 = dectree1.predict(x_test)

    # Confusion Matrix
    CM_test1 = confusion_matrix(y_test, y_test_pred1)
   
    return [dectree1.score(x_test, y_test), CM_test1[0][1] / (CM_test1[0][0]+CM_test1[0][1]), CM_test1[1][0] / (CM_test1[1][1]+CM_test1[1][0])]

In [4]:
rawData = pd.read_csv('cleanData.csv')
df = pd.DataFrame(rawData)

for index, row in df.iterrows():
    if pd.isna(row['Ratio Transfers']):
        df.loc[index, 'Ratio Transfers'] = 0

df

,Unnamed: 0,Unnamed: 0.1,WID,Label,Eth Balance,Txn Count,Exchange,ERC 20 Txn,ERC 20 Txn In,ERC 20 Txn Out,ERC721 Txn Out,ERC721 Txn In,ERC721 Txn,Ratio Transfers
0,113,113,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,Bittrex,6090.819190,11133470.0,Y,10002.0,5065.0,4937.0,0.0,18.0,18.0,1.03
1,179,179,0x2faf487a4414fe77e2327f0bf4ae2a264a776ad2,FTX Exchange,221468.466722,3575544.0,Y,10000.0,4404.0,5596.0,0.0,0.0,0.0,0.79
2,29,29,0x8f22f2063d253846b53609231ed80fa571bc0c8f,Binance 13,1.157549,545203.0,Y,10000.0,19.0,9981.0,0.0,0.0,0.0,0.00
3,70,70,0xc56fefd1028b0534bfadcdb580d3519b5586246e,Bitfinex 11,95263.263235,1.0,Y,6.0,6.0,0.0,0.0,0.0,0.0,1000.00
4,45,45,0x06a0048079ec6571cd1b537418869cde6191d42d,Binance 29,37913.561938,5.0,Y,242.0,241.0,1.0,0.0,0.0,0.0,241.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,839,839,0xfba17aca0797f72bd8152c5e519100c0c4caf0ad,NaN,18.701764,924.0,N,419.0,215.0,204.0,0.0,0.0,0.0,1.05
948,840,840,0xfbbe05954c6b138999548171c272a1b109d89471,NaN,0.197973,214.0,N,23.0,14.0,9.0,0.0,0.0,0.0,1.56
949,841,841,0xfefa8d5093ffd962a094fdba722a2c30d1c7a822,NaN,0.858955,28.0,N,0.0,0.0,0.0,0.0,4.0,4.0,1.00
950,842,842,0xff64a8933e05c9d585ab72db95d207ebee9be5a8,NaN,30106.172270,22.0,N,104.0,104.0,0.0,0.0,0.0,0.0,1000.00


In [20]:
FPR={}
FNR={}
CA={}
testset = ["Txn Count", 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out', 'Ratio Transfers']
for i in range(10):
    for j in range(len(testset)):
        k = j + 1
        while k <= len(testset):
            y = pd.DataFrame(df['Exchange'])
            x = pd.DataFrame(df[testset[j:k]])
            tree = classify(x, y)
            if i == 0:
                CA[str(testset[j:k])] = tree[0]
                FPR[str(testset[j:k])] = tree[1]
                FNR[str(testset[j:k])] = tree[2]
            else:
                CA[str(testset[j:k])] += tree[0]
                FPR[str(testset[j:k])] += tree[1]
                FNR[str(testset[j:k])] += tree[2]
            k += 1
for i in CA:
    print(i, "data")
    print("Average classification accuracy for", ": ", CA[i]/10)
    print("Average false positive rate", ": ", FPR[i]/10)
    print("Average false negative rate", ": ", FNR[i]/10)
    print("=====================================")

['Txn Count'] data
Average classification accuracy for :  0.8418848167539267
Average false positive rate :  0.09960444556154184
Average false negative rate :  0.21197666915974467
['Txn Count', 'ERC 20 Txn'] data
Average classification accuracy for :  0.8795811518324609
Average false positive rate :  0.07169672346251191
Average false negative rate :  0.1639092041970792
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In'] data
Average classification accuracy for :  0.8832460732984293
Average false positive rate :  0.07456836694066463
Average false negative rate :  0.15891150216769107
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out'] data
Average classification accuracy for :  0.8958115183246074
Average false positive rate :  0.07707052784717605
Average false negative rate :  0.1301131902995693
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out', 'Ratio Transfers'] data
Average classification accuracy for :  0.8989528795811518
Average false positive rate :  0.11154150966014

## Decision Tree Conclusion
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out', 'Ratio Transfers'] has the best classification accuracy, so we will use all 5 predictors for our Decision Tree model later.


## Random Forest

In [13]:
def rf(pred, response):

    predTrain, predTest, responseTrain, responseTest = train_test_split(pred, response, test_size=0.2, random_state=42)

    rforest = RandomForestClassifier(n_estimators=300, max_depth=9)
    rforest.fit(predTrain, responseTrain.values.ravel())

    yTestPred = rforest.predict(predTest)

    # Plot the Confusion Matrix 
    CM_test1 = confusion_matrix(responseTest, yTestPred)

    return [rforest.score(predTest, responseTest), CM_test1[0][1] / (CM_test1[0][0]+CM_test1[0][1]), CM_test1[1][0] / (CM_test1[1][1]+CM_test1[1][0])]


In [28]:
FPR={}
FNR={}
CA={}
testset = ["Txn Count", 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out', 'Ratio Transfers']
for i in range(10):
    for j in range(len(testset)):
        k = j + 1
        while k <= len(testset):
            y = pd.DataFrame(df['Exchange'])
            x = pd.DataFrame(df[testset[j:k]])
            forest = rf(x, y)
            if i == 0:
                CA[str(testset[j:k])] = forest[0]
                FPR[str(testset[j:k])] = forest[1]
                FNR[str(testset[j:k])] = forest[2]
            else:
                CA[str(testset[j:k])] += forest[0]
                FPR[str(testset[j:k])] += forest[1]
                FNR[str(testset[j:k])] += forest[2]
            k += 1
for i in CA:
    print(i, "data")
    print("Average classification accuracy for", ": ", CA[i]/10)
    print("Average false positive rate", ": ", FPR[i]/10)
    print("Average false negative rate", ": ", FNR[i]/10)
    
    print("=====================================")

['Txn Count'] data
Average classification accuracy for :  0.8230366492146597
Average false positive rate :  0.1442307692307692
Average false negative rate :  0.2160919540229885
['Txn Count', 'ERC 20 Txn'] data
Average classification accuracy for :  0.9078534031413612
Average false positive rate :  0.09807692307692308
Average false negative rate :  0.08505747126436783
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In'] data
Average classification accuracy for :  0.8942408376963351
Average false positive rate :  0.13749999999999998
Average false negative rate :  0.067816091954023
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out'] data
Average classification accuracy for :  0.9214659685863875
Average false positive rate :  0.1182692307692308
Average false negative rate :  0.03103448275862069
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out', 'Ratio Transfers'] data
Average classification accuracy for :  0.9094240837696335
Average false positive rate :  0.14038461538461536

## Random Forest Conclusion
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out'] has the best classification accuracy, so we will use these 4 predictors for our Random Forest model later.



## KNN

In [10]:
def splitData(a1, a2):
    xTrain, xTest, yTrain, yTest = train_test_split(a1, a2, test_size=0.2)
    return [xTrain, xTest, yTrain, yTest]
    
def knn(a1, a2):
    knn = KNeighborsClassifier(n_neighbors=6)
    y = pd.DataFrame(df[a2])
    x = pd.DataFrame(df[a1])
    datasplit = splitData(x,y)
    knn.fit(datasplit[0], datasplit[2].values.ravel())
    y_pred = knn.predict(datasplit[1])
    CM_test1 = confusion_matrix(datasplit[3], y_pred)
    return [metrics.accuracy_score(datasplit[3], y_pred), CM_test1[0][1] / (CM_test1[0][0]+CM_test1[0][1]), CM_test1[1][0] / (CM_test1[1][1]+CM_test1[1][0])]

In [12]:
FPR={}
FNR={}
CA={}
testset = ["Txn Count", 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out', 'Ratio Transfers']
for i in range(10):
    for j in range(len(testset)):
        k = j + 1
        while k <= len(testset):
            y = ['Exchange']
            x = testset[j:k]
            forest = knn(x, y)
            if i == 0:
                CA[str(testset[j:k])] = forest[0]
                FPR[str(testset[j:k])] = forest[1]
                FNR[str(testset[j:k])] = forest[2]
            else:
                CA[str(testset[j:k])] += forest[0]
                FPR[str(testset[j:k])] += forest[1]
                FNR[str(testset[j:k])] += forest[2]
            k += 1
for i in CA:
    print(i, "data")
    print("Average classification accuracy for", ": ", CA[i]/10)
    print("Average false positive rate", ": ", FPR[i]/10)
    print("Average false negative rate", ": ", FNR[i]/10)
    print("=====================================")

['Txn Count'] data
Average classification accuracy for :  0.8293193717277486
Average false positive rate :  0.08466346027563751
Average false negative rate :  0.25042372901859505
['Txn Count', 'ERC 20 Txn'] data
Average classification accuracy for :  0.8853403141361257
Average false positive rate :  0.08409071603529213
Average false negative rate :  0.14779170830416025
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In'] data
Average classification accuracy for :  0.8602094240837698
Average false positive rate :  0.10099956177582507
Average false negative rate :  0.17653770001480346
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out'] data
Average classification accuracy for :  0.893193717277487
Average false positive rate :  0.0706859080537893
Average false negative rate :  0.14438452426216009
['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out', 'Ratio Transfers'] data
Average classification accuracy for :  0.8753926701570679
Average false positive rate :  0.08881536318949

## KNN Conclusion

['Txn Count', 'ERC 20 Txn', 'ERC 20 Txn In', 'ERC 20 Txn Out'] has the best classification accuracy, so we shall use those predictors on our KNN model.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=696215f1-387b-461d-aa4c-0ccf546cb4fd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>